1.爬取房价不同地方的外面殿
2.爬取学校面积不同地方的外卖店
3.爬起爬取之后需要对照周边是否有大型购物中心

In [1]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from browsermobproxy import Server
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver import ActionChains
from selenium.webdriver.remote.webdriver import NoSuchElementException
from selenium.webdriver.remote.errorhandler import ElementClickInterceptedException
from selenium.common.exceptions import StaleElementReferenceException

import jsonpath
import re
import time
import pandas as pd
import numpy as np
import json
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
# 运行代理服务
PATH = "C:\\Users\\caoni\\Desktop\\projecet\\python\\scrapy\\eleme\\browsermob-proxy-2.1.4\\bin\\browsermob-proxy.bat"
server = Server(PATH,{"port":8739})
server.start()
proxy = server.create_proxy()

In [4]:
chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument("../Selenium_drive/chromedriver.exe")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
# 反反爬虫：掩盖selenium浏览器指纹，可在控制台通过window.navigator.webdriver查看得false
chrome_options.add_experimental_option('excludeSwitches', ['enable-automation'])
# chrome升级至v59以后，可用如下headless的方式使得不显示浏览器界面
"""
chrome_options.add_argument('--headless')
# chrome_options.add_argument('--disable-gpu')
"""
chrome_options.add_argument('--ignore-certificate-errors') 
#对游览器设置代理
# chrome_options.add_argument('--proxy-server={0}'.format(proxy.proxy))
driver = webdriver.Chrome(chrome_options=chrome_options)

driver.get("https://tb.ele.me/wow/msite/act/login")

# 登录页面操作
# 通过 id 切换框架
driver.switch_to.frame('alibaba-login-box')

# 获取并自动填充手机号
driver.find_element(By.XPATH,'//input[@id="fm-sms-login-id"]').send_keys("18368028196")
# 同意协议并发送验证码
driver.find_element(By.ID,"fm-agreement-checkbox").click()
# 离开iframe
# driver.switch_to.default_content()
driver.find_element(By.XPATH,"/html/body/div/div/div[1]/div/form/div[3]/div[2]/a").click()

c:\Users\caoni\.conda\envs\WebCrawle\lib\site-packages\ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


In [4]:
# 地理位置模拟设置 selemue4 
def changeLocation(longitude,latitude):
    # driver = webdriver.Chrome()
    Map_coordinates = dict({
        "latitude": latitude, # 纬度
        "longitude": longitude, # 经度
        "accuracy": 100 # 定位准确性
        })
    driver.execute_cdp_cmd("Emulation.setGeolocationOverride", Map_coordinates)
    # 每次修改地址之后刷新网页
    driver.get("https://h5.ele.me")
    time.sleep(3)
    return driver.find_element(By.CLASS_NAME,"address-txt")
  

### 开始爬取网页

In [5]:
    # 鼠标滚轮事件下滑网页
def down(_):
        driver.find_element(By.TAG_NAME,"html").send_keys(Keys.END)

In [ ]:
# driver.get("https://h5.ele.me")

In [6]:
# 导入地区的经纬度
longAndlatitude = pd.read_csv("./wenzhou.csv")

longAndlatitude.iloc[0][0]

27.6724873

In [ ]:
coordinate = 0

In [9]:
for coordinate in range(coordinate,len(longAndlatitude)):
    long = longAndlatitude.iloc[coordinate][0]
    atitude = longAndlatitude.iloc[coordinate][1]
    # 先给定一个CSV列表
    fileName = f"{atitude}_{long}.csv"
    open(f"./coordinate_store/{fileName}","w",encoding="utf-8-sig").write("brandName,point,sell(M),getTime(Min),distance(KM),deliverStart,deliverFee,vipCoupon,Location,URL\n")
    grab_homepage(fileName,atitude,long)

120.2935867_27.7956562.csv,8
120.3797302_27.7935734.csv,5
120.4658585_27.7914371.csv,205
120.5517883_27.7909126.csv,205
120.6380997_27.7870121.csv,205
120.7241974_27.784708.csv,205
120.8092194_27.7925644.csv,42
120.863327_27.804985.csv,5
120.2283554_27.8685303.csv,5
120.2959061_27.872263.csv,5
120.382103_27.8701725.csv,5
120.382103_27.8701725.csv,5
120.4682999_27.8680305.csv,10
120.5544815_27.8658352.csv,43
120.6406479_27.8635826.csv,205
120.7268143_27.8612804.csv,103
120.8129654_27.8589211.csv,205
120.884346_27.8634567.csv,25
120.2443924_27.9311256.csv,5
120.3005066_27.9409218.csv,5
120.3846817_27.9465961.csv,5
120.4707413_27.944622.csv,11
120.5569839_27.9424191.csv,205
120.643219_27.9401627.csv,205
120.7294388_27.9378529.csv,125
120.8136368_27.9337769.csv,205
120.8886337_27.91362.csv,34
120.3397751_27.9976368.csv,5
120.389122_28.0217075.csv,5
120.4731979_28.0212135.csv,33
120.5588455_28.0185528.csv,205
120.6457977_28.0040169.csv,205
120.7305679_27.9996376.csv,205
120.7916641_27.98853

In [ ]:
# 逻辑：先让网页向下滑到底部

In [7]:
def grab_homepage(fileName,atitude,long):
    flag = 1
    while 1:
        try:
            if flag !=1 and driver.find_element(By.CLASS_NAME,"list_status").text != "正在加载更多...":
            # 解包
                pack = proxy.har
            # get shop URL
                urls = []
                for i, j in enumerate(pack['log']['entries']):
                    if 'shopId=' in j['response']['content'].get('text', ''):
                        text = pack['log']['entries'][i]['response']['content']['text']
                        if text.startswith('('): continue
                        text = json.loads(text)
                        urls.extend(text['data']['data']['frontend_page_shop_list_recommend']['fields']['items'])
                print(f"{fileName},{len(urls)}")
                local = driver.find_element(By.CLASS_NAME,"address-txt").text
                
                for i in urls:
                    # get data
                    url = i["fields"]["restaurant"]["scheme"]
                    brandName = i["fields"]["restaurant"]["name"]
                    point = i["fields"]["restaurant"]["rating"]
                    sell = i["fields"]["restaurant"]["recentOrderNum"]
                    # 部分商家会采用快递发货所以 orderLeadTime 字段可能会不存在,,
                    try:
                        getTime = i["fields"]["restaurant"]["orderLeadTime"]
                    except KeyError:
                        getTime = "NaN"
                    try:
                        distance = i["fields"]["restaurant"]["distance"]
                    except KeyError:
                        distance = "NaN"
                    deliverStart = i["fields"]["restaurant"]["piecewiseAgentFee"]["rules"][0]["price"]
                    try:
                        deliverFee = i["fields"]["restaurant"]["piecewiseAgentFee"]["rules"][0]["fee"]
                    except KeyError:
                        deliverFee = i["fields"]["restaurant"]["piecewiseAgentFee"]["description"]
                    try:
                        vipCoupon = "-".join(jsonpath.jsonpath(i["fields"]["restaurant"]["tagRegion"]["foldingSupportTags"],"$..text"))
                    except KeyError:
                        vipCoupon = "NaN"
                    open(f"./coordinate_store/{fileName}","a",encoding="utf-8-sig").write(f"{brandName},{point},{sell},{getTime},{round(float(distance)/1000,1)},{deliverStart},{deliverFee},{vipCoupon},{local},{url}\n")
                    # print(f"{brandName},{point},{sell},{getTime},{round(float(distance)/1000,1)},{deliverStart},{deliverFee},{vipCoupon},{local},{url}")
                # exist loop
                return True
                break
                # stop proxy server
                # server.stop()

            else:

                down(_)

                if flag:
                # 开始抓包
                # 抓包/刷新包中的内容
                    proxy.new_har("ele", options={'captureHeaders': True, 'captureContent': True})
                # 给定经纬度 并刷新网页
                    Location = changeLocation(atitude,long)
                # 点击网页任意元素让down生效
                    driver.find_element(By.CLASS_NAME,"nav-ele").click()
                    flag = 0
        except StaleElementReferenceException:
            # print("")
            if grab_homepage(fileName,atitude,long):
                break
        except NoSuchElementException:
            if driver.find_element(By.CLASS_NAME,"text-content").text == "你的网络饿晕过去了，请刷新重试":
                # 解包
                pack = proxy.har
            # get shop URL
                urls = []
                for i, j in enumerate(pack['log']['entries']):
                    if 'shopId=' in j['response']['content'].get('text', ''):
                        text = pack['log']['entries'][i]['response']['content']['text']
                        if text.startswith('('): continue
                        text = json.loads(text)
                        urls.extend(text['data']['data']['frontend_page_shop_list_recommend']['fields']['items'])
                print(f"{fileName},{len(urls)}")
                local = driver.find_element(By.CLASS_NAME,"address-txt").text
                
                for i in urls:
                    # get data
                    url = i["fields"]["restaurant"]["scheme"]
                    brandName = i["fields"]["restaurant"]["name"]
                    point = i["fields"]["restaurant"]["rating"]
                    sell = i["fields"]["restaurant"]["recentOrderNum"]
                    # 部分商家会采用快递发货所以 orderLeadTime 字段可能会不存在,,
                    try:
                        getTime = i["fields"]["restaurant"]["orderLeadTime"]
                    except KeyError:
                        getTime = "NaN"
                    try:
                        distance = i["fields"]["restaurant"]["distance"]
                    except KeyError:
                        distance = "NaN"
                    deliverStart = i["fields"]["restaurant"]["piecewiseAgentFee"]["rules"][0]["price"]
                    try:
                        deliverFee = i["fields"]["restaurant"]["piecewiseAgentFee"]["rules"][0]["fee"]
                    except KeyError:
                        deliverFee = i["fields"]["restaurant"]["piecewiseAgentFee"]["description"]
                    try:
                        vipCoupon = "-".join(jsonpath.jsonpath(i["fields"]["restaurant"]["tagRegion"]["foldingSupportTags"],"$..text"))
                    except KeyError:
                        vipCoupon = "NaN"
                    open(f"./coordinate_store/{fileName}","a",encoding="utf-8-sig").write(f"{brandName},{point},{sell},{getTime},{round(float(distance)/1000,1)},{deliverStart},{deliverFee},{vipCoupon},{local},{url}\n")
                    # print(f"{brandName},{point},{sell},{getTime},{round(float(distance)/1000,1)},{deliverStart},{deliverFee},{vipCoupon},{local},{url}")
                # exist loop
                return True
                break
                # stop proxy server
                # server.stop()


In [13]:
# 分析抓取过来的homePage文件夹内的文件 合并所有csv文件 去重 

In [12]:
fileNamelist = os.listdir("./coordinate_store/")
allFileADD = pd.concat([pd.read_csv(f"./coordinate_store/{fileName}") for fileName in fileNamelist])
allFileADD.reset_index(drop=True)
back_allFile = allFileADD

In [18]:
allFileADD.drop_duplicates(["brandName","point","sell(M)"], inplace=True)

In [20]:
allFileADD.to_csv('data.csv', index=False, encoding='utf-8-sig')

##### 去重2 去除上次爬取的数据

In [252]:
latest_data = pd.read_csv("./已经爬取.csv")

delete_data = pd.read_csv("./duplicate_value.csv")

print(len(delete_data))



# delete_data[delete_data['brandName'].isin(["华莱士·全鸡汉堡(郭溪景德店)"])&delete_data['point'].isin([4.5])]
for b,p in zip(latest_data.brandName,latest_data.point):
    d = delete_data[delete_data['brandName'].isin([b])&delete_data['point'].isin([float(p)])]
    delete_data = delete_data.drop(d.index)

2696


In [ ]:
get_data

In [256]:
# 保存为CSV
print(len(delete_data))
delete_data.to_csv("./again_duplicates.csv",encoding="utf-8-sig")

2332


## CODE TEST 

In [5]:
# 读取
rewriteInfo = pd.read_csv("./data.csv")

In [6]:
driver.get(rewriteInfo.URL[0])

In [9]:
def hide(element):
    js = '''var a = document.getElementsByClassName('J_MIDDLEWARE_FRAME_WIDGET');
    for (i in a) {
        a[i].style = "display: none";
    }'''
    driver.execute_script(js)

In [10]:
def click(element):
    element[0].click()

In [28]:
def doIfExist(element, *f):
    for f in f:
        if element: f(element)

In [45]:
def clickOrSkip(element):
    return not element or element[-1].click() or time.sleep(2)

In [39]:
index = 180

In [25]:
index = 0
T = t = time.time()

In [ ]:
rewriteInfo[:index].to_csv('data1.csv', index=False, encoding='utf-8-sig')

In [14]:
import csv

In [26]:
f = open('data1.csv', 'w', encoding='utf-8-sig', newline='\n')
writer = csv.writer(f)
writer.writerow(['', "Merchant address", "Type", "openTime"])
f.close()

In [19]:
f.close()

In [ ]:
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By



In [54]:
from selenium.common.exceptions import TimeoutException

In [ ]:
EC.presence_of_all_elements_located

In [73]:
for index in range(index, len(rewriteInfo)):
    driver.get(rewriteInfo.URL[index])
    time.sleep(5)
    doIfExist(driver.find_elements(By.TAG_NAME, 'iframe'), hide)
    doIfExist(driver.find_elements(By.CLASS_NAME, 'arrow'), click)
    doIfExist(driver.find_elements(By.CLASS_NAME, 'closeIcon__a68d961f'), click)
    try:
        WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'tab__item-text')))[-1].click()
    except TimeoutException:
        f = open('data1.csv', 'a', encoding='utf-8-sig', newline='\n')
        writer = csv.writer(f)
        writer.writerow([index])
        f.close()
        continue
    
    doIfExist(driver.find_elements(By.CLASS_NAME, 'action__right'), click, lambda : time.sleep(2))
    try:
        shop_add = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME,"shop__info__name-location-location"))).text
        shop_type, opening_time = driver.find_elements(By.CLASS_NAME,"shop__info__category-item")
        shop_type = shop_type.text.replace('商家品类：', '')
        opening_time = opening_time.text.replace('营业时间：', '')
        f = open('data1.csv', 'a', encoding='utf-8-sig', newline='\n')
        writer = csv.writer(f)
        writer.writerow([index, shop_add, shop_type, opening_time])
        f.close()
    except TimeoutException:
        f = open('data1.csv', 'a', encoding='utf-8-sig', newline='\n')
        writer = csv.writer(f)
        writer.writerow([index, 'wait'])
        f.close()
        continue
    if not index % 20:
        print(f'已完成: {index}项, 用时{time.time() - t:.2f}s, 总共用时: {(time.time() - T) / 60:.2f} min')
        t = time.time()

已完成: 480项, 用时424.96s, 总共用时: 76.96 min
已完成: 500项, 用时136.63s, 总共用时: 79.24 min
已完成: 520项, 用时134.04s, 总共用时: 81.47 min
已完成: 540项, 用时143.65s, 总共用时: 83.87 min
已完成: 560项, 用时135.92s, 总共用时: 86.13 min
已完成: 580项, 用时139.87s, 总共用时: 88.46 min
已完成: 600项, 用时135.20s, 总共用时: 90.72 min
已完成: 620项, 用时136.59s, 总共用时: 92.99 min
已完成: 640项, 用时144.53s, 总共用时: 95.40 min
已完成: 660项, 用时133.60s, 总共用时: 97.63 min
已完成: 680项, 用时134.17s, 总共用时: 99.87 min
已完成: 700项, 用时135.03s, 总共用时: 102.12 min
已完成: 720项, 用时137.09s, 总共用时: 104.40 min
已完成: 740项, 用时150.96s, 总共用时: 106.92 min
已完成: 760项, 用时134.96s, 总共用时: 109.17 min


KeyboardInterrupt: 

In [ ]:
def replace_plus(r_list,r_str,r_character):
    n_r_str = str(r_str)
    for i in r_list:
        for j in range(0,len(r_str)):
            if i == r_str[j]:
                n_r_str = n_r_str.replace(i,r_character)
    return n_r_str
               
replace_plus(["起","¥"],"¥23.98起","")
def clear(unequal,num,clearValue):
    """
    unequal:
        + 清除大于num的数
        -  清除小于num的数
        = 清楚等于num的数
    """
    sortedList = sorted(clearValue)

    if unequal == "+":
        for i in range(len(sortedList)):
            if sortedList[i] > num:
                return sortedList[i::]

    if unequal == "-":
        for i in range(len(sortedList)):
            if sortedList[i] > num:
                return sortedList[:i]
    if unequal == "=":
        return list(filter((num).__ne__,sortedList))


In [301]:
# 进一步精简Q中的数组
def condenseQ(Q):
    condenseQ = 0
    for i in Q:
        if driver.find_elements(By.CLASS_NAME,"menuItem")[-1].find_element(By.CLASS_NAME,"menuItem--info-title").text in str(i):
            condenseQ = json.loads(i["response"]["content"]["text"])["data"]["resultMap"]["menu"]["itemGroups"]
            break
    return condenseQ

# 从json获取所有商品价格(商品价格+打包费)
def allPrice(condenseQ):
    if condenseQ:
        # 暂时找不到计算用户平均消费的良好数据暂时传入数组
        allPriceL = []
        for i in condenseQ:
            for j in i["items"][1::]:
                packageFee = jsonpath.jsonpath(j,"$..packageFee")[-1]
                if jsonpath.jsonpath(j,"$..useCouponPrice"):
                    commodityPrice = jsonpath.jsonpath(j,"$..useCouponPrice")[-1]
                    commodityPrice = replace_plus(["起","¥"],commodityPrice,"")
                else:
                    commodityPrice = jsonpath.jsonpath(j,"$..price")[-1]
                statement = float(packageFee) + float(commodityPrice)
                # print(statement)
                allPriceL.append(statement)
        # print(allPrice)
        return allPriceL
    else:
        return "SOME EEROR"

In [303]:
for index in range(1572,len(rewriteInfo)):
    driver.get(urls.URL[index])
    # 开始抓包
    # 抓包/刷新包中的内容
    proxy.new_har("ele", options={'captureHeaders': True, 'captureContent': True})
    # 出现元素不可点击的错误请拉长sleep时间
    time.sleep(3)
    # 使down函数生效
    try:
        driver.find_elements(By.CLASS_NAME,"tab__item-text")[0].click()
    except ElementClickInterceptedException:
        # 发现滑块验证
        if len(driver.find_elements(By.TAG_NAME,"iframe")) == 2:
            fuck_verification()
        else:
            driver.find_element(By.CLASS_NAME,"shop-critical-status-container").find_element(By.CLASS_NAME,"arrow").click()
            driver.find_elements(By.CLASS_NAME,"tab__item-text")[0].click()
            time.sleep(1)
    except IndexError:
        shopInfo =  "NaN","NaN","NaN","NaN",
        rewriteInfo.loc[index, ["allPrice","Merchant address", "Type", "openTime"]] = shopInfo 
        rewriteInfo.to_csv("./coordinate_store/allFile.csv",encoding="utf-8-sig")
        continue

    down(_)
    time.sleep(3)
    pack = proxy.har
    pack = pack["log"]["entries"]
    # Q 中保存POST时"mtop.alsc.waimai.store.miniapp.store.detail.body.query" 的请求
    Q = []
    for i in range(0,len(pack)):
        if "mtop.alsc.waimai.store.miniapp.store.detail.body.query" in json.dumps(pack[i]["request"]["queryString"]):
            Q.append(pack[i])

    # 将精简完成的Q传入下面的函数获取商店内的所有价格
    if Q:
        try :
            C = condenseQ(Q)
        except IndexError:
            continue
        allPriceList = allPrice(C)
        newC = clear("=",0.0,allPriceList)
    else:
        print(f"Q IS NULL .FUCK!!! index:{index}")
        # null_price.append(index)
        newC = ""
    # 价格获取完毕之后 获取商家信息
    driver.find_elements(By.CLASS_NAME,"tab__item-text")[-1].click()
    time.sleep(4)

    # 商家地址
    shop_add = driver.find_element(By.CLASS_NAME,"shop__info__name-location-location").text

    # 
    #     
    #     # shop_add = driver.find_element(By.CLASS_NAME,"shop__info__name-location-location").text
    #     shop_add = driver.find_element(By.CLASS_NAME,"shop__info__name-location-location").text
    # except NoSuchElementException:
    #         shop_add = driver.find_element(By.CLASS_NAME,"shop__info__name-location-location").text



    # 商品类型
    shop_type = driver.find_elements(By.CLASS_NAME,"shop__info__category-item")[0]  
    shop_type = shop_type.text.replace("商家品类：","")

    # 经营时间
    opening_time = driver.find_elements(By.CLASS_NAME,"shop__info__category-item")[1]
    opening_time = opening_time.text.replace("营业时间：","")

    if newC:
        RE_price = "-".join([str(i) for i in newC])
    else:
        RE_price = "NaN"
        

    shopInfo =  RE_price,shop_add,shop_type,opening_time,
    rewriteInfo.loc[index, ["allPrice","Merchant address", "Type", "openTime"]] = shopInfo 
    # rewriteInfo.to_csv("./coordinate_store/allFile.csv",encoding="utf-8-sig")



Q IS NULL .FUCK!!! index:1519
Q IS NULL .FUCK!!! index:1572


KeyboardInterrupt: 

In [288]:
shop_add = driver.find_element(By.CLASS_NAME,"shop__info__name-location-location").text
shop_add

'浙江省温州市瓯海区梧田街道梧慈路1号第三间(不作为拆迁凭证)'

### 滑块验证解决

In [65]:
# 通过 网页框架数量 切换框架
def fuck_verification():
    driver.switch_to.frame(1)
    fuck = driver.find_element(By.ID,"nc_1_n1z")
    action =ActionChains(driver)
    action.click_and_hold(fuck)
    action.move_by_offset(300,0)
    action.release().perform()
    driver.switch_to.default_content()

In [ ]:
driver.find_elements(By.TAG_NAME,"iframe")

In [304]:
rewriteInfo.to_csv("./备用已抓取1500.csv",encoding="utf-8-sig")

In [ ]:
# 经纬度距离计算 /分析
from geopy.distance import geodesic
print(geodesic((27.982025,120.807333), (28.1367, 120.557371)).km) #计算两个坐标直线距离
# print(geodesic((27.982025,120.807333), (27.983374,120.805307)).km) #计算两个坐标直线距离 #计算两个坐标直线距离

In [ ]:
[120.557371, 28.1367][::-1]

In [ ]:
for i in urls:
    # json中获取数据
    url = i["fields"]["restaurant"]["scheme"]
    brandName = i["fields"]["restaurant"]["name"]
    point = i["fields"]["restaurant"]["rating"]
    sell = i["fields"]["restaurant"]["recentOrderNum"]
    getTime = i["fields"]["restaurant"]["orderLeadTime"]
    distance = i["fields"]["restaurant"]["distance"]
    deliverStart = i["fields"]["restaurant"]["piecewiseAgentFee"]["rules"][0]["price"]
    try:
        deliverFee = i["fields"]["restaurant"]["piecewiseAgentFee"]["rules"][0]["fee"]
    except KeyError:
        deliverFee = i["fields"]["restaurant"]["piecewiseAgentFee"]["description"]
    try:
        vipCoupon = "-".join(jsonpath.jsonpath(i["fields"]["restaurant"]["tagRegion"]["foldingSupportTags"],"$..text"))
    except KeyError:
        vipCoupon = "NULL"
    open("./shopInfo.csv","a",encoding="utf-8-sig").write(f"{brandName},{point},{sell},{getTime},{round(float(distance)/1000,1)},{deliverStart},{deliverFee},{vipCoupon},{'/'.join(list(Location))},{url}\n")

In [ ]:
# 获取URL
urls = []
for i, j in enumerate(c['log']['entries']):
    if 'shopId=' in j['response']['content'].get('text', ''):
        text = c['log']['entries'][i]['response']['content']['text']
        if text.startswith('('): continue
        text = json.loads(text)
        urls.extend(text['data']['data']['frontend_page_shop_list_recommend']['fields']['items'])
len(urls)


In [ ]:
# json中获取数据
url = urls[0]["fields"]["restaurant"]["scheme"]
brandName = urls[0]["fields"]["restaurant"]["name"]
point = urls[0]["fields"]["restaurant"]["rating"]
sell = urls[0]["fields"]["restaurant"]["recentOrderNum"]
getTime = urls[0]["fields"]["restaurant"]["orderLeadTime"]
distance = urls[0]["fields"]["restaurant"]["distance"]
deliverStart = urls[0]["fields"]["restaurant"]["piecewiseAgentFee"]["rules"][0]["price"]
deliverFee = urls[0]["fields"]["restaurant"]["piecewiseAgentFee"]["rules"][0]["fee"]
# ensure_ascii=False 不转义非英文字符串
# vipCoupon = json.dumps(urls[0]["fields"]["restaurant"]["tagRegion"]["foldingSupportTags"],ensure_ascii=False)
vipCoupon = "-".join(jsonpath.jsonpath(urls[0]["fields"]["restaurant"]["tagRegion"]["foldingSupportTags"],"$..text"))

open("./shopInfo.csv","w",encoding="utf-8-sig").write(f"{brandName},{point},{sell},{getTime},{round(float(distance)/1000,1)},{deliverStart},{deliverFee},{vipCoupon},{'/'.join(list(Location))},{url}")

In [ ]:
# 停止代理服务
server.stop()

In [ ]:
# 通过标签抓取 (舍弃)
storeInfo = driver.find_elements(By.CLASS_NAME,"card-takeaway-big")
Nlocation = "，".join(list(Location))
for i in storeInfo:
    brandName = i.find_element(By.CLASS_NAME,"card-takeaway__title").text
    try:
        point = i.find_element(By.CLASS_NAME,"card-takeaway__store-point").text
    except NoSuchElementException:
        point = "new shop"
    try:
        sell = i.find_element(By.CLASS_NAME,"card-takeaway__store-sell").text
    except NoSuchElementException:
        sell = "new shop"
    getTime = i.find_element(By.CLASS_NAME,"card-takeaway__store-distance").text
    distance = i.find_element(By.CLASS_NAME,"card-takeaway__store-time").text
    deliverStart = i.find_element(By.CLASS_NAME,"card-takeaway__delivery-start").text
    deliverFee = i.find_element(By.CLASS_NAME,"card-takeaway__delivery-fee").text
    vipCoupon = "-".join(i.find_element(By.CLASS_NAME,"card-takeaway__tags-one").text.split("\n"))
    # print(vipCoupon)
    open("./shopInfo.csv","a",encoding="utf-8-sig").write(f"{brandName},{point},{sell},{getTime},{distance},{deliverStart},{deliverFee},{vipCoupon},{Nlocation}\n")
# 抓取标签(舍弃)

for i in storeInfo:
    try:
        a = i.split("\n")
        brandName = a[0]
        point = a[1]
        sell = a[3]

        getTime,distance = a[4].split("分钟")

        regex = r"(\D*)¥(\d*)(.*)"

        test_str = a[5]

        matches = re.match(regex, test_str)

        deliverStart = matches.group(2)
        deliverFee = matches.group(3)
        vipCoupon = "-".join(a[7::])
        Nlocation = "，".join(list(Location))
        
        open("./shopInfo.csv","w",encoding="utf-8-sig").write(f"{brandName},{point},{sell},{getTime},{distance},{deliverStart},{deliverFee},{vipCoupon},{Nlocation}\n")
    except ValueError:
        print(i)
        # open("./shopInfo.csv","a",encoding="utf-8-sig").write(f"{a}\n")

120.73825,